In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt


In [ ]:
import os, sys
sys.path.append(os.path.abspath('..'))

from src.tagseg.data.acdc_dataset import AcdcDataSet

In [ ]:
acdc = AcdcDataSet(filepath='../data/01_raw/acdc-training')

In [ ]:
dataset = acdc.load()

In [ ]:
acdc._describe()

In [ ]:
from kedro.extras.datasets.pickle import PickleDataSet

In [ ]:
from kedro.config import ConfigLoader

In [ ]:
conf_paths = ["../conf/base", "../conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("parameters*", "parameters*/**")

In [ ]:
conf_catalog

In [ ]:
# Find out where to store .pt file based on params
acdc_features = acdc._describe()
tagged = acdc_features['tagged']
only_myo = acdc_features['only_myo']

filename: str = 'acdc_dataset'
filename += '_tagged' if tagged else '_cine'
filename += '_only_myo' if only_myo else ''
filename += '.pt'

In [ ]:
pds = PickleDataSet(filepath=f'../data/05_model_input/{filename}')

In [ ]:
pds.exists()

In [ ]:
pds.save(dataset)

In [ ]:
ds = pds.load()